# 一 从tfrecord文件读取数据

In [1]:
import numpy as np
import tensorflow as tf
import os
from PIL import Image 
import matplotlib.pyplot as plt 

#### 定义读取函数

In [2]:
def read_and_decode(filename,num): # 读入train.tfrecords
    filename_queue = tf.train.string_input_producer([filename])#生成一个queue队列

    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)#返回文件名和文件
    features = tf.parse_single_example(serialized_example,
                                       features={
                                           'label': tf.FixedLenFeature([], tf.int64),
                                           'img_raw' : tf.FixedLenFeature([], tf.string),
                                       })#将image数据和label取出来

    image = tf.decode_raw(features['img_raw'], tf.uint8)
    image = tf.cast(image, tf.float32)    # 解码之后转数据类型 
    image = tf.reshape(image, [48, 24])
    #image = tf.reshape(image, [128, 128, 3])  #reshape为128*128的3通道图片
    #img = tf.cast(img, tf.float32) * (1. / 255) - 0.5 #在流中抛出img张量
    label = tf.cast(features['label'], tf.int32) #在流中抛出label张量
    image_batch, label_batch = tf.train.shuffle_batch([image,label], batch_size=1, capacity=200,
                                                 min_after_dequeue=100, num_threads=2)

    with tf.Session() as sess: #开始一个会话
        new_img = np.zeros((num, 1152))
        new_lab = np.zeros((num, 1))

        init = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
        sess.run(init)
        
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(sess=sess, coord=coord)
        for count in range(num):

            image, label = sess.run([image_batch, label_batch])  # 在会话中取出image和label
            img = image.reshape([48, 24]) 
            img = img.astype(np.uint8)  # PIL保存时，必须是整数
            if num == 1:
                coord.request_stop()
                coord.join(threads)
                return img, label
            else:
                image = image.reshape(1152)
                image = image / 255
                new_img[count, :] = image
                new_lab[count, :] = label
               # if count % 1000 == 0:
                   # print(count)
        coord.request_stop()  
        coord.join(threads)
    return new_img, new_lab
    

# 二.设计卷积神经网络结构并利用卷积神经网络对汉字和字母数字分别进行训练

#### 读取训练集和测试集数据

In [3]:
[X_train, y_train] = read_and_decode('train.tfrecords', 10856)
[X_val, y_val] = read_and_decode('validation.tfrecords', 3619)

#### 转换标签数据为one-hot类型

In [4]:
def dense_to_one_hot(labels_dense, num_classes=100):
    labels_dense = labels_dense.astype(np.uint8)   # 更换数据类型
    num_labels = labels_dense.shape[0]
    index_offset = np.arange(num_labels) * num_classes
    labels_one_hot = np.zeros((num_labels, num_classes))
    labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
    return labels_one_hot
#
y_train = dense_to_one_hot(y_train, 100)
y_val = dense_to_one_hot(y_val, 100)

In [5]:
import numpy as np
import tensorflow as tf
import random

In [6]:
FLAGS = tf.app.flags.FLAGS
# Basic model parameters.
tf.app.flags.DEFINE_string('my_list', './cnn',"""存放模型的目录""")
tf.app.flags.DEFINE_string('my_parameters', 'mnist',"""模型的名称""")

In [7]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.InteractiveSession(config=config)   # 启动计算图

In [8]:
print(X_train.shape)
print(y_train.shape)
print(y_val.shape)

(10856, 1152)
(10856, 100)
(3619, 100)


#### 构建卷积神经网络

In [9]:
#定义一个函数，用于初始化所有的权值 W
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

#定义一个函数，用于初始化所有的偏置项 b
def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)
  
#定义一个函数，用于构建卷积层
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

#定义一个函数，用于构建池化层
def max_pool(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1], padding='SAME')

In [10]:
def chooseone(image, label, batchsize):
    im = np.empty((batchsize, 1152))
    la = np.empty((batchsize, 36))
    for i in range(batchsize):
        a = random.randint(0, 3474)
        im[i] = image[a]
        la[i] = label[a]
    return im, la

In [11]:
X_ = tf.placeholder(tf.float32, [None, 1152])
y_ = tf.placeholder(tf.float32, [None, 100])

# 把X转为卷积所需要的形式
X = tf.reshape(X_, [-1, 48, 24, 1])
# 第一层卷积：3×3×1卷积核32个 [3，3，1，32],h_conv1.shape=[-1, 48, 24, 32],学习32种特征
W_conv1 = weight_variable([3,3,1,32])
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(X, W_conv1) + b_conv1)

# 第一个pooling 层[-1, 48, 24, 32]->[-1, 24, 12, 32]
h_pool1 = max_pool(h_conv1)

# 第二层卷积：5×5×32卷积核64个 [3，3，32，64],h_conv2.shape=[-1, 24, 12, 64]
W_conv2 = weight_variable([3,3,32,64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)

# 第二个pooling 层,[-1, 24, 12, 64]->[-1, 14, 6, 64] 
h_pool2 = max_pool(h_conv2)

# 第三层卷积
W_conv3 = weight_variable([3, 3, 64, 96])
b_conv3 = bias_variable([96])
h_conv3 = tf.nn.relu(conv2d(h_pool2, W_conv3) + b_conv3)
h_pool3 = max_pool(h_conv3)

# flatten层，[-1, 6, 3, 96]->[-1, 6*3*96],即每个样本得到一个6*3*96维的样本
h_pool2_flat = tf.reshape(h_pool3, [-1, 6*3*96])

# fc1
W_fc1 = weight_variable([6*3*96, 512])
b_fc1 = bias_variable([512])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# dropout: 输出的维度和h_fc1一样，只是随机部分值被值为零
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# 输出层
W_fc2 = weight_variable([512, 100])
b_fc2 = bias_variable([100])
y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)


# 1.损失函数：cross_entropy
cross_entropy = -tf.reduce_sum(y_ * tf.log(y_conv))
# 2.优化函数：AdamOptimizer
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

# 3.预测准确结果统计
#　预测值中最大值（１）即分类结果，是否等于原始标签中的（１）的位置。argmax()取最大值所在的下标
z = tf.argmax(y_conv, 1)
q = tf.argmax(y_, 1)
correct_prediction = tf.equal(z, q)  
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# 保存
saver = tf.train.Saver(max_to_keep=2)

sess.run(tf.global_variables_initializer())
writer = tf.summary.FileWriter(FLAGS.my_list, sess.graph)

ckpt = tf.train.latest_checkpoint(FLAGS.my_list)
step = 0
if ckpt:
    saver.restore(sess=sess, save_path=ckpt)
    step = int(ckpt[len(os.path.join(FLAGS.my_list, FLAGS.my_parameters)) + 1:])


INFO:tensorflow:Restoring parameters from ./cnn/mnist-900


#### 随机批次载入数据

In [12]:
def min_next_batch_tfr(image, label, num=50): 
    images = np.zeros((num, 1152))
    labels = np.zeros((num, 100))
    for i in range(num):
        temp = random.randint(0, 10855)
        images[i, :] = image[temp]
        labels[i, :] = label[temp]

    return images, labels

In [13]:
import tensorflow as tf

# 开启线程
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)

'''
# 训练
for i in range(1000):   ## 20000
    batch = min_next_batch_tfr(X_train, y_train, 50)
    if i % 100 == 0:   ## 1000
        train_accuracy, loss = sess.run([accuracy, cross_entropy], feed_dict={
            X_: batch[0], y_: batch[1], keep_prob: 1})
        print("step %d, validating accuracy %g, loss is %g" % (i, train_accuracy, loss))
        ckptname=os.path.join(FLAGS.my_list, FLAGS.my_parameters)
        saver.save(sess,ckptname,global_step=i)
    train_step.run(feed_dict={X_: batch[0], y_: batch[1], keep_prob: 0.5})
'''

    
# 利用测试集对卷积神经网络进行检测，并得到识别正确率
     
print("test accuracy %g" % accuracy.eval(feed_dict={ X_: X_val , y_: y_val, keep_prob: 1.0}))

# prediction
y_conv = np.argmax(y_conv.eval(feed_dict={X_: X_val, y_: y_val, keep_prob: 1.0}), 1)
y_conv = y_conv.reshape(-1, 1)
coord.request_stop() 
coord.join(threads)

# validation
y_val = np.argmax(y_val, 1)
y_val = y_val.reshape(-1)

# 拼接  validation + prediction
test = np.column_stack((y_val, y_conv))
test = test[np.lexsort(test[:, ::-1].T)] 
    
# 计算召回率
for count in range(34):
    for i in range(test.shape[0]):
        j = test[i][0]  # 找出validation 的 label
        if i != 0:
            j1 = test[i-1][0]
            if j1 != j:
                test_temp = test[:i]
                correct_prediction = np.equal(test_temp[:, 0], test_temp[:, 1])
                accuracy = np.mean(correct_prediction)
                accuracy = accuracy.astype('float32')
                if j1 > 10:
                    j1 = chr(j1)
                    print('%s的召回率为%f'%(j1, accuracy))
                else:
                    print('%d的召回率为%f'%(j1, accuracy))

                test = test[i:]
                break

# z的召回率
test_temp = test
correct_prediction = np.equal(test_temp[:, 0], test_temp[:, 1])
accuracy = np.mean(correct_prediction)
accuracy = accuracy.astype('float32')
print('z的召回率为%f'% accuracy)

test accuracy 0.937552
0的召回率为0.918519
1的召回率为0.964029
2的召回率为0.935780
3的召回率为0.955357
4的召回率为0.934783
5的召回率为0.764706
6的召回率为0.882979
7的召回率为0.728155
8的召回率为0.822034
9的召回率为0.788991
A的召回率为0.995575
B的召回率为0.933333
C的召回率为1.000000
D的召回率为0.555556
E的召回率为0.983936
F的召回率为0.969231
G的召回率为0.967213
H的召回率为0.983333
J的召回率为1.000000
K的召回率为0.963636
L的召回率为1.000000
M的召回率为0.985294
N的召回率为0.966667
P的召回率为0.921569
Q的召回率为0.811321
R的召回率为1.000000
S的召回率为0.964706
T的召回率为0.864865
U的召回率为0.941176
V的召回率为1.000000
W的召回率为1.000000
X的召回率为0.995951
Y的召回率为0.993827
z的召回率为0.967213
